In [2]:
import os
from langchain_community.utilities import SQLDatabase

# OpenAI API 키 설정 (이 예제에서는 사용되지 않음)
api_key = os.getenv("OPENAI_API_KEY")

# SQLite 데이터베이스 경로
db_path = "/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db"

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 데이터베이스 방언 출력 (예: sqlite)
print(db.dialect)

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

# SQL 쿼리 실행 및 결과 출력
result = db.run("SELECT * FROM Artist LIMIT 10;")
print(result)

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]


In [26]:
import pprint
from operator import itemgetter
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


from langchain.globals import set_verbose
from langchain.globals import set_debug
set_verbose(False)
set_debug(False)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

answer_prompt = PromptTemplate.from_template(
    """ 
사용자 질문에 친절하게 답변해 주세요.
만약, 데이터베이스와 관련된 질문인 경우에는 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.

- 데이터베이스 관련 질문일 경우 : 
질문: {question}
SQL 쿼리: {query}
SQL 결과: {result}
답변: 
"""
)


# StrOutputParser 설정
output_parser = StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt | llm | output_parser
)


response = chain.invoke({"question": "몇명의 직원이 있어?"})
generated_sql = write_query.invoke({"question": "몇명의 직원이 있어?"})
sql_result = db.run(generated_sql)

print("1. 생성된 SQL Query :")
print(generated_sql)
print("\n")

print("2. SQL Query 실행 결과 :")
print(sql_result)
print("\n")

print("3. 최종 답변 :")
pprint.pprint(response)
print("\n")


1. 생성된 SQL Query :
SELECT COUNT(EmployeeId) AS TotalEmployees
FROM Employee


2. SQL Query 실행 결과 :
[(8,)]


3. 최종 답변 :
'현재 회사에는 8명의 직원이 있습니다.'




In [15]:
from pydantic import BaseModel, EmailStr, ValidationError
from typing import List

# 사용자 모델 정의
class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    friends: List[str]

# 유효한 데이터 예시
data_1 = {
    "id": 1,
    "name": "이영우",
    "email": "duddn6464@naver.com",
    "friends": ["최창우", "박지성"]
}

# 유효하지 않은 데이터 예시
data_2 = {
    "id": "1",
    "name": "이영우",
    "email": "duddn6464",
    "friends": "최창우, 박지성"
}

try:
    user = User(**data_1)
    print(f"{data_1} 생성완료")
except ValidationError as e:
    print("Validation error:", e)

try:
    user = User(**data_2)
    print(f"{data_2}생성완료")
except ValidationError as e:
    print("Validation error:", e)

{'id': 1, 'name': '이영우', 'email': 'duddn6464@naver.com', 'friends': ['최창우', '박지성']} 생성완료
Validation error: 2 validation errors for User
email
  value is not a valid email address: The email address is not valid. It must have exactly one @-sign. [type=value_error, input_value='duddn6464', input_type=str]
friends
  Input should be a valid list [type=list_type, input_value='최창우, 박지성', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/list_type


In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
set_verbose(False)
set_debug(False)

prompt = ChatPromptTemplate.from_template("나에게 {topic} 관련한 농담을 말해줘.")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain = prompt | model | output_parser
chain.invoke({"topic": "사과"})

'사과가 나무에서 떨어지면... 사과했다!'

In [18]:
chain.batch([{"topic": "사과"}, {"topic": "커피"}])

['사과가 나무에서 떨어지면서 외치는 소리는 뭐야? "애플!"',
 '커피는 삶의 유일한 즐거움이자, 새벽부터 밤늦게까지 나를 살려주는 최고의 친구야. 커피 없이는 나의 인생은 흑백사진처럼 지루하고 단조로울 거야. 커피는 나의 작은 행복이자, 큰 힘이야!']

In [19]:
for s in chain.stream({"topic": "사과"}):
    print(s)


사
과
가
 학
교
에
 가
면
 무
슨
 과
목
을
 듣
게
 될
까
요
?


"
과
일
과
학
"
입니다
!



In [20]:
# RunnablePassthrough
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

chain = prompt | model | output_parser
question = "what is langsmith"
context = "langsmith is a testing and observability platform built by the langchain team"
chain.invoke({"question": question, "context": context})

'Langsmith is a testing and observability platform built by the langchain team.'

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever
) | chain

retrieval_chain.invoke({"question": "what is langsmith"})

In [ ]:
from langchain_core.runnables import RunnableParallel
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})
parallel_chain.invoke({"question": "what is langsmith"})

In [ ]:
from langchain_core.runnables import RunnableParallel
prompt = ChatPromptTemplate.from_template("""{question}""")
simple_chain = prompt | model | output_parser
parallel_chain = RunnableParallel({
    "retrieved_answer": retrieval_chain,
    "simple_answer": simple_chain
})
result = parallel_chain.invoke({"question": "what is langsmith"})
print(result)

In [25]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 사용자 정의 함수들
def length_function(text):  # 텍스트의 길이를 반환 함수
    return len(text)

def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)

def multiple_length_function(_dict):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])

# prompt template
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
model = ChatOpenAI()

# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke({"input_1": "bar", "input_2": "gah"})
print(result) 

3 + 9 = 12
